## Amazon SageMaker Feature Store: Introduction to Feature Store

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/prepare_data|sm-feature_store_introduction|sm-feature_store_introduction.ipynb)

---

This notebook demonstrates how to get started with Feature Store, create feature groups, and ingest data into them. These feature groups are stored in your Feature Store.

Feature groups are resources that contain metadata for all data stored in your Feature Store. A feature group is a logical grouping of features, defined in the feature store to describe records. A feature group’s definition is composed of a list of feature definitions, a record identifier name, and configurations for its online and offline store. 

### Overview
1. Set up
2. Creating a feature group
3. Ingest data into a feature group

### Prerequisites
This notebook uses sagemaker_core SDK and `Python 3 (Data Science)` kernel. This notebook works with Studio, Jupyter, and JupyterLab. 

#### Library dependencies:
* `sagemaker_core`
* `numpy`
* `pandas`

#### Role requirements:
**IMPORTANT**: You must attach the following policies to your execution role:
* `AmazonS3FullAccess`
* `AmazonSageMakerFeatureStoreAccess`

![policy](images/feature-store-policy.png)

In [40]:
!pip uninstall pydantic sagemaker-core -y
!pip install pip --upgrade --quiet
!pip uninstall sagemaker-core -y
!pip install sagemaker-core --upgrade

Found existing installation: pydantic 2.9.0
Uninstalling pydantic-2.9.0:
  Successfully uninstalled pydantic-2.9.0
Found existing installation: sagemaker-core 1.0.3
Uninstalling sagemaker-core-1.0.3:
  Successfully uninstalled sagemaker-core-1.0.3
  Using cached sagemaker_core-1.0.3-py3-none-any.whl.metadata (4.9 kB)
  Using cached pydantic-2.9.0-py3-none-any.whl.metadata (146 kB)
Using cached sagemaker_core-1.0.3-py3-none-any.whl (377 kB)
Using cached pydantic-2.9.0-py3-none-any.whl (434 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gluonts 0.13.7 requires pydantic~=1.7, but you have pydantic 2.9.0 which is incompatible.
langchain-community 0.2.11 requires langchain<0.3.0,>=0.2.12, but you have langchain 0.2.5 which is incompatible.


In [ ]:
!pip uninstall pydantic sagemaker-core -y
!pip install --upgrade pip -q
!pip install pydantic==2.7.0
!pip install sagemaker-core

In [41]:
!pip freeze | grep pydantic

pydantic==2.9.0
pydantic_core==2.23.2


In [42]:
import pandas as pd
import numpy as np
import io
from sagemaker_core.helper.session_helper import get_execution_role, Session
sagemaker_session = Session()
region = "us-east-1"
REGION_NAME = region if region else SM_SESSION._region_name
role = get_execution_role()
s3_bucket_name  = sagemaker_session.default_bucket()
prefix = "sagemaker-featurestore-introduction"

### Set up

### Inspect your data
In this notebook example we ingest synthetic data. We read from `./data/feature_store_introduction_customer.csv` and `./data/feature_store_introduction_orders.csv`.

In [43]:
customer_data = pd.read_csv("data/feature_store_introduction_customer.csv")
orders_data = pd.read_csv("data/feature_store_introduction_orders.csv")

In [44]:
customer_data.head()

,customer_id,city_code,state_code,country_code
0,573291,1,49,2
1,109382,2,40,2
2,828400,3,31,2
3,124013,4,5,2


In [45]:
orders_data.head()

,customer_id,order_id,order_status,store_id
0,573291,4132,1,303
1,109382,5724,0,201
2,828400,1942,0,431
3,124013,6782,1,213


Below is an illustration on the steps the data goes through before it is ingested into a Feature Store. In this notebook, we illustrate the use-case where you have data from multiple sources and want to store them independently in a feature store. Our example considers data from a data warehouse (customer data), and data from a real-time streaming service (order data). 

### Create a feature group

We first start by creating feature group names for customer_data and orders_data. Following this, we create two Feature Groups, one for `customer_data` and another for `orders_data`

In [46]:
from time import gmtime, strftime, sleep

customers_feature_group_name = "customers-feature-group-"  + strftime("%d-%H-%M-%S", gmtime())
orders_feature_group_name = "orders-feature-group-" + strftime("%d-%H-%M-%S", gmtime())

In [47]:
customers_feature_group_name

'customers-feature-group-06-11-54-09'

Instantiate a FeatureGroup object for customers_data and orders_data. 

In [48]:
from sagemaker_core.shapes import FeatureDefinition

CustomerFeatureDefinitions =[FeatureDefinition(feature_name='customer_id',feature_type='Integral'),
                             FeatureDefinition(feature_name='city_code',feature_type='Integral'),
                             FeatureDefinition(feature_name='state_code',feature_type='Integral'),
                             FeatureDefinition(feature_name='country_code',feature_type='Integral'),
                             FeatureDefinition(feature_name='EventTime',feature_type='Fractional')]

OrderFeatureDefinitions = [FeatureDefinition(feature_name='customer_id',feature_type='Integral'),
                           FeatureDefinition(feature_name='order_id', feature_type='Integral'),
                           FeatureDefinition(feature_name='order_status', feature_type='Integral'),
                           FeatureDefinition(feature_name='store_id', feature_type='Integral'),
                           FeatureDefinition(feature_name='EventTime', feature_type='Fractional')]

In [49]:
customers_feature_group_name

'customers-feature-group-06-11-54-09'

In [50]:
import time

current_time_sec = int(round(time.time()))

record_identifier_feature_name = "customer_id"

Append `EventTime` feature to your data frame. This parameter is required, and time stamps each data point.

In [51]:
customer_data["EventTime"] = pd.Series([current_time_sec] * len(customer_data), dtype="float64")
orders_data["EventTime"] = pd.Series([current_time_sec] * len(orders_data), dtype="float64")

Load feature definitions to your feature group. 

Below we call create to create two feature groups, customers_feature_group and orders_feature_group respectively

In [52]:
from sagemaker_core.shapes import OnlineStoreConfig,OfflineStoreConfig,S3StorageConfig
from sagemaker_core.resources import FeatureGroup

FeatureGroup.create(
    feature_group_name= customers_feature_group_name,
    #s3_uri=f"s3://{s3_bucket_name}/{prefix}",
    record_identifier_feature_name= record_identifier_feature_name,
    event_time_feature_name="EventTime",
    role_arn=role,
    online_store_config = OnlineStoreConfig(enable_store_config=True),
    feature_definitions = CustomerFeatureDefinitions
    #offline_store_config = OfflineStoreConfig(s3_storage_config = S3StorageConfig(s3_uri = f"s3://{s3_bucket_name}/{prefix}"))
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:10                                                                                   │
│                                                                                                  │
│    7 │   record_identifier_feature_name= record_identifier_feature_name,                         │
│    8 │   event_time_feature_name="EventTime",                                                    │
│    9 │   role_arn=role,                                                                          │
│ ❱ 10 │   online_store_config = OnlineStoreConfig(enable_store_config=True),                      │
│   11 │   feature_definitions = CustomerFeatureDefinitions                                        │
│   12 │   #offline_store_config = OfflineStoreConfig(s3_storage_config = S3StorageConfig(s3_ur    │
│   13 )                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pydantic/main.py:175 in __init__                         │
│                                                                                                  │
│    172 │   │   """                                                                               │
│    173 │   │   # `__tracebackhide__` tells pytest and some other tools to omit this function fr  │
│    174 │   │   __tracebackhide__ = True                                                          │
│ ❱  175 │   │   self.__pydantic_validator__.validate_python(data, self_instance=self)             │
│    176 │                                                                                         │
│    177 │   # The following line sets a flag that we use to determine when `__init__` gets overr  │
│    178 │   __init__.__pydantic_base_init__ = True  # pyright: ignore[reportFunctionMemberAccess  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValidationError: 1 validation error for OnlineStoreConfig
enable_store_config
  Extra inputs are not permitted [type=extra_forbidden, input_value=True, input_type=bool]
    For further information visit https://errors.pydantic.dev/2.7/v/extra_forbidden

In [39]:
from sagemaker_core.resources import FeatureGroup
FeatureGroup.create(
    feature_group_name= orders_feature_group_name,
    record_identifier_feature_name= record_identifier_feature_name,
    event_time_feature_name="EventTime",
    role_arn=role,
    online_store_config= OnlineStoreConfig(enable_store_config=True),
    feature_definitions = OrderFeatureDefinitions,
    offline_store_config =  OfflineStoreConfig(s3_storage_config = S3StorageConfig(s3_uri = f"s3://{s3_bucket_name}/{prefix}"))
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:7                                                                                    │
│                                                                                                  │
│    4 │   record_identifier_feature_name= record_identifier_feature_name,                         │
│    5 │   event_time_feature_name="EventTime",                                                    │
│    6 │   role_arn=role,                                                                          │
│ ❱  7 │   online_store_config= OnlineStoreConfig(enable_store_config=True),                       │
│    8 │   feature_definitions = OrderFeatureDefinitions,                                          │
│    9 │   offline_store_config =  OfflineStoreConfig(s3_storage_config = S3StorageConfig(s3_ur    │
│   10 )                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pydantic/main.py:175 in __init__                         │
│                                                                                                  │
│    172 │   │   """                                                                               │
│    173 │   │   # `__tracebackhide__` tells pytest and some other tools to omit this function fr  │
│    174 │   │   __tracebackhide__ = True                                                          │
│ ❱  175 │   │   self.__pydantic_validator__.validate_python(data, self_instance=self)             │
│    176 │                                                                                         │
│    177 │   # The following line sets a flag that we use to determine when `__init__` gets overr  │
│    178 │   __init__.__pydantic_base_init__ = True  # pyright: ignore[reportFunctionMemberAccess  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValidationError: 1 validation error for OnlineStoreConfig
enable_store_config
  Extra inputs are not permitted [type=extra_forbidden, input_value=True, input_type=bool]
    For further information visit https://errors.pydantic.dev/2.7/v/extra_forbidden

To confirm that your FeatureGroup has been created we use `DescribeFeatureGroup` and `ListFeatureGroups` APIs to display the created FeatureGroup.

In [ ]:
customers_feature_group.describe()

In [ ]:
orders_feature_group.describe()

In [ ]:
sagemaker_session.boto_session.client(
    "sagemaker", region_name=region
).list_feature_groups()  # We use the boto client to list FeatureGroups

In [ ]:
def check_feature_group_status(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group to be Created")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    print(f"FeatureGroup {feature_group.name} successfully created.")


check_feature_group_status(customers_feature_group)
check_feature_group_status(orders_feature_group)

### Add metadata to a feature

We can add searchable metadata fields to FeatureGroup features by using the `UpdateFeatureMetadata` API. The currently supported metadata fields are `description` and `parameters`.

In [79]:
from sagemaker_core.resources import FeatureMetadata
from sagemaker_core.shapes import FeatureParameter

customers_feature_group.update_feature_metadata(
    feature_name="customer_id",
    description="The ID of a customer. It is also used in orders_feature_group.",
    parameter_additions=[FeatureParameter("idType", "primaryKey")],
)

customers_feature_metadata = FeatureMetadata(feature_group_name = customers_feature_group_name, feature_name = "customer_id",
                                             description= 'The ID of a customer. It is also used in orders_feature_group.',
                                             parameters = [FeatureParameter(key="idType", value="primaryKey")]
                                             )

To view feature metadata, we can use `get` method to display that feature.

In [95]:
customers_feature_metadata_3= FeatureMetadata.get(feature_group_name = customers_feature_group_name, feature_name = "customer_id")

In [97]:
customers_feature_metadata_3.update(description="This is a test 3",parameter_additions=[FeatureParameter(key="idType", value="primaryKey")])

[09/03/24 17:07:08] INFO     INFO:sagemaker_core.main.resources:Updating feature_metadata         ]8;id=883813;file:///opt/conda/lib/python3.10/site-packages/sagemaker_core/main/resources.py\resources.py]8;;\:]8;id=983103;file:///opt/conda/lib/python3.10/site-packages/sagemaker_core/main/resources.py#9635\9635]8;;\
                             resource.                                                                             

FeatureMetadata(feature_group_name='customers-feature-group-03-16-59-32', feature_name='customer_id', feature_group_arn='arn:aws:sagemaker:us-east-1:774297356213:feature-group/customers-feature-group-03-16-59-32', feature_type='Integral', creation_time=datetime.datetime(2024, 9, 3, 17, 0, 51, 605000, tzinfo=tzlocal()), last_modified_time=datetime.datetime(2024, 9, 3, 17, 7, 8, 194000, tzinfo=tzlocal()), description='This is a test 3', parameters=[FeatureParameter(key='idType', value='primaryKey')])

Feature metadata fields are searchable. We use `search` API to find features with metadata that matches some search criteria.

In [53]:
sagemaker_session.boto_session.client("sagemaker", region_name=region).search(
    Resource="FeatureMetadata",
    SearchExpression={
        "Filters": [
            {
                "Name": "FeatureGroupName",
                "Operator": "Contains",
                "Value": "customers-feature-group-",
            },
            {"Name": "Parameters.idType", "Operator": "Equals", "Value": "primaryKey"},
        ]
    },
)  # We use the boto client to search

{'Results': [{'FeatureMetadata': {'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:774297356213:feature-group/customers-feature-group-30-03-11-20',
    'FeatureGroupName': 'customers-feature-group-30-03-11-20',
    'FeatureName': 'customer_id',
    'FeatureType': 'Integral',
    'CreationTime': datetime.datetime(2024, 8, 30, 3, 11, 22, tzinfo=tzlocal()),
    'LastModifiedTime': datetime.datetime(2024, 8, 30, 3, 11, 56, tzinfo=tzlocal()),
    'Description': 'The ID of a customer. It is also used in orders_feature_group.',
    'Parameters': [{'Key': 'idType', 'Value': 'primaryKey'}]}}],
 'ResponseMetadata': {'RequestId': '2bdda8bf-fdaa-4218-9884-e96d747e936a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2bdda8bf-fdaa-4218-9884-e96d747e936a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '486',
   'date': 'Fri, 30 Aug 2024 03:40:40 GMT'},
  'RetryAttempts': 0}}

### Ingest data into a feature group

We can put data into the FeatureGroup by using the `PutRecord` API. It will take < 1 minute to ingest data.

In [54]:
customers_feature_group.ingest(data_frame=customer_data, max_workers=3, wait=True)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 customers_feature_group.ingest(data_frame=customer_data, max_workers=3, wait=True)           │
│   2                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/pydantic/main.py:828 in __getattr__                      │
│                                                                                                  │
│    825 │   │   │   │   │   │   return super().__getattribute__(item)  # Raises AttributeError i  │
│    826 │   │   │   │   │   else:                                                                 │
│    827 │   │   │   │   │   │   # this is the current error                                       │
│ ❱  828 │   │   │   │   │   │   raise AttributeError(f'{type(self).__name__!r} object has no att  │
│    829 │   │                                                                                     │
│    830 │   │   def __setattr__(self, name: str, value: Any) -> None:                             │
│    831 │   │   │   if name in self.__class_vars__:                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'FeatureGroup' object has no attribute 'ingest'

In [ ]:
orders_feature_group.ingest(data_frame=orders_data, max_workers=3, wait=True)

Using an arbitrary customer record ID, 573291 we use `get_record` to check that the data has been ingested into the feature group.

In [ ]:
customer_id = 573291
sample_record = sagemaker_session.boto_session.client(
    "sagemaker-featurestore-runtime", region_name=region
).get_record(
    FeatureGroupName=customers_feature_group_name, RecordIdentifierValueAsString=str(customer_id)
)

In [ ]:
sample_record

We use `batch_get_record` to check that all data has been ingested into two feature groups by providing customer IDs.

In [ ]:
all_records = sagemaker_session.boto_session.client(
    "sagemaker-featurestore-runtime", region_name=region
).batch_get_record(
    Identifiers=[
        {
            "FeatureGroupName": customers_feature_group_name,
            "RecordIdentifiersValueAsString": ["573291", "109382", "828400", "124013"],
        },
        {
            "FeatureGroupName": orders_feature_group_name,
            "RecordIdentifiersValueAsString": ["573291", "109382", "828400", "124013"],
        },
    ]
)

In [ ]:
all_records

### Add features to a feature group

If we want to update a FeatureGroup that has done the data ingestion, we can use the `UpdateFeatureGroup` API and then re-ingest data by using the updated dataset.

In [ ]:
customers_feature_group.update(
    feature_additions=[{'feature_name': 'Email', 'feature_type': 'String'},{'feature_name': 'name', 'feature_type': 'String'}]
)

Verify the FeatureGroup has been updated successfully or not.

In [ ]:
def check_last_update_status(feature_group):
    last_update_status = feature_group.get("LastUpdateStatus")["Status"]
    while last_update_status == "InProgress":
        print("Waiting for FeatureGroup to be updated")
        time.sleep(5)
        last_update_status = feature_group.get("LastUpdateStatus")
    if last_update_status == "Successful":
        print(f"FeatureGroup {feature_group.name} successfully updated.")
    else:
        print(
            f"FeatureGroup {feature_group.name} updated failed. The LastUpdateStatus is"
            + str(last_update_status)
        )


check_last_update_status(customers_feature_group)

Inspect the new dataset.

In [ ]:
customer_data_updated = pd.read_csv("data/feature_store_introduction_customer_updated.csv")

In [ ]:
customer_data_updated.head()

Append `EventTime` feature to your data frame again.

In [ ]:
customer_data_updated["EventTime"] = pd.Series(
    [current_time_sec] * len(customer_data), dtype="float64"
)

Ingest the new dataset.

In [ ]:
## need to see how to ingest data in new SDK

customers_feature_group.ingest(data_frame=customer_data_updated, max_workers=3, wait=True)

Use `batch_get_record` again to check that all updated data has been ingested into `customers_feature_group` by providing customer IDs.

In [ ]:
updated_customers_records = sagemaker_session.boto_session.client(
    "sagemaker-featurestore-runtime", region_name=region
).batch_get_record(
    Identifiers=[
        {
            "FeatureGroupName": customers_feature_group_name,
            "RecordIdentifiersValueAsString": ["573291", "109382", "828400", "124013"],
        }
    ]
)

In [ ]:
updated_customers_records

### Clean up
Here we remove the Feature Groups we created. 

In [ ]:
customers_feature_group.delete()
orders_feature_group.delete()

### Next steps

In this notebook you learned how to quickly get started with Feature Store and now know how to create feature groups, and ingest data into them.

For an advanced example on how to use Feature Store for a Fraud Detection use-case, see [Fraud Detection with Feature Store](https://sagemaker-examples.readthedocs.io/en/latest/sagemaker-featurestore/sagemaker_featurestore_fraud_detection_python_sdk.html).

For detailed information about Feature Store, see the [Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store.html).


### Programmers note

In this notebook we used a variety of different API calls. Most of them are accessible through the Python SDK, however some only exist within `boto3`. You can invoke the Python SDK API calls directly on your Feature Store objects, whereas to invoke API calls that exist within `boto3`, you must first access a boto client through your boto and sagemaker sessions: e.g. `sagemaker_session.boto_session.client()`.

Below we list API calls used in this notebook that exist within the Python SDK and ones that exist in `boto3` for your reference. 

#### Python SDK API Calls
* `describe()`
* `ingest()`
* `delete()`
* `create()`
* `load_feature_definitions()`
* `update()`
* `update_feature_metadata()`
* `describe_feature_metadata()`

#### Boto3 API Calls
* `list_feature_groups()`
* `get_record()`
* `batch_get_record()`
* `search()`


## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/prepare_data|sm-feature_store_introduction|sm-feature_store_introduction.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/prepare_data|sm-feature_store_introduction|sm-feature_store_introduction.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/prepare_data|sm-feature_store_introduction|sm-feature_store_introduction.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/prepare_data|sm-feature_store_introduction|sm-feature_store_introduction.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/prepare_data|sm-feature_store_introduction|sm-feature_store_introduction.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/prepare_data|sm-feature_store_introduction|sm-feature_store_introduction.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/prepare_data|sm-feature_store_introduction|sm-feature_store_introduction.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/prepare_data|sm-feature_store_introduction|sm-feature_store_introduction.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/prepare_data|sm-feature_store_introduction|sm-feature_store_introduction.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/prepare_data|sm-feature_store_introduction|sm-feature_store_introduction.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/prepare_data|sm-feature_store_introduction|sm-feature_store_introduction.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/prepare_data|sm-feature_store_introduction|sm-feature_store_introduction.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/prepare_data|sm-feature_store_introduction|sm-feature_store_introduction.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/prepare_data|sm-feature_store_introduction|sm-feature_store_introduction.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/prepare_data|sm-feature_store_introduction|sm-feature_store_introduction.ipynb)
